In [1]:
def price_lookup(ticker: str) -> str:
    return f"Latest price for {ticker.upper()} is $720.50."

def news_headlines(ticker: str, n: int) -> str:
    return f"Top {n} headlines for {ticker.upper()}: [Headline1, Headline2, ...]"

def stat_ratios(ticker: str) -> str:
    return f"P/E: 35.2, P/S: 12.7, ROE: 18.4% for {ticker.upper()}"

In [23]:
from langchain.llms import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="google/flan-t5-xl",  # Choose appropriate model
    temperature=0.3,
    max_length=512
)

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


In [24]:
print(type(llm))

<class 'langchain_community.llms.huggingface_endpoint.HuggingFaceEndpoint'>


In [25]:
TOOL_DESCRIPTIONS = """
Available tools:
1. price_lookup(ticker: str): Returns the latest stock price.
2. news_headlines(ticker: str, n: int): Returns recent news headlines for the stock.
3. stat_ratios(ticker: str): Returns P/E, P/S, ROE ratios for the stock.
"""

In [26]:
def router_node(state):
    user_input = state["query"]
    prompt = f"{TOOL_DESCRIPTIONS}\n\nGiven this user query: \"{user_input}\", decide the tool to call and return it as JSON.\nExample:\n{{'tool': 'stat_ratios', 'args': {{'ticker': 'AAPL'}}}}"
    
    response = llm.invoke(prompt)  # `.predict` for ChatOpenAI, `.invoke` or `()` for others
    try:
        tool_call = eval(response)  # or use json.loads() with stricter formatting
    except:
        tool_call = {"tool": None, "args": {}}
    
    return {"tool_call": tool_call, "query": user_input}


In [27]:
def tool_executor(state):
    call = state["tool_call"]
    tool_name = call.get("tool")
    args = call.get("args", {})

    if tool_name == "price_lookup":
        result = price_lookup(**args)
    elif tool_name == "news_headlines":
        result = news_headlines(**args)
    elif tool_name == "stat_ratios":
        result = stat_ratios(**args)
    else:
        result = "Tool not found or invalid arguments."
    
    return {"tool_response": result, "query": state["query"]}


In [28]:
def answer_composer(state):
    prompt = f"""
Given the user query: "{state['query']}" and tool response: "{state['tool_response']}",
compose a final answer to return to the user.
"""
    return {"final_answer": llm(prompt)}


In [29]:
from langgraph.graph import StateGraph, END

builder = StateGraph(dict)

builder.add_node("router", router_node)
builder.add_node("tool_executor", tool_executor)
builder.add_node("composer", answer_composer)

builder.set_entry_point("router")
builder.add_edge("router", "tool_executor")
builder.add_edge("tool_executor", "composer")
builder.add_edge("composer", END)

graph = builder.compile()

In [30]:
result = graph.invoke({"query": "Give me the P/E ratio for NVDA."})
print(result["final_answer"])

AttributeError: 'InferenceClient' object has no attribute 'post'